In [1]:
import pandas as pd
import string
from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 23 days


# BW1484 Core I Output

In [2]:
k_ref = 1.00020

### Denovo ouput file processor

In [57]:
import re
import os
import numpy as np
import csv

class DenovoProcessor(object):
    """DenovoProcessor class: processes an output file from a Denovo calculation.
       It can read a file, print that information to csv, and do some processing
       for highlights."""
    
    def __init__(self):
        """Set information we'll use in processing a Denovo output file"""    
        # Output variables that are in the header / problem specification
        self.keys = ['problem_name', 'num cells I', 'num cells J', 'num cells K', 'cells', 'mesh ',
                     'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'blocks', 'num_groups', 
                     'num upscatter groups', 'num_sets', 'domains', 'Pn_order', 'azimuthals_octant', 
                     'polars_octant', 'eq_set', 'DoF', 'tolerance', 'L2_tolerance', 'k_tolerance',
                     'up_tol', 'eigen_solver', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
                     'relax_weight', 'ml_Pn', 'ml_az', 'ml_oct']
        # Information about timing and processor distribution
        self.time_keys = ['procs-per-node-x', 'procs-per-node-y', 'walltime']
        # Iteration information that is printed during execution
        self.iter_dict = {'Iterations =' : [], 'Eigenvalue Iteration   ' : [],
                          'relative error in k-eff of ' : [],
                          'absolute error in k-eff of ' : []}  
        self.k = {'eigenvalue of' : []}
                
    def parse_file(self, out_file):
        """Read the values for the keys from out_file"""
        # Initalize values to match keys to None
        self.vals = {key: None for key in keys}
        self.time_vals = {key: None for key in time_keys}
    
        # Set flags and keys to use for items with duplicate entries
        ml_flag = False
        ml_keys = ['Pn_order', 'azimuthals_octant', 'polars_octant']
        ml_doubles = {key: None for key in ml_keys}
        ml_cnt = 0
        up_db = False
        doubles = {'tolerance' : None}
    
        # open the file
        if os.path.isfile(out_file):
            f = open(out_file, 'r')
        else:
            raise IOError('could not find file', full_out_file)
   
        # Process the file
        for line in f:
    
            # check for the duplicate items before processing everything else
            if ml_flag == True: 
                if ml_cnt != 3:
                    for ml_key in ml_doubles:
                        if re.search(ml_key, line):
                            if ml_doubles[ml_key] == None:
                                data = line.split(ml_key)
                                ml_doubles[ml_key] = data[1].strip() 
                                ml_cnt = ml_cnt + 1
                else:
                    ml_flag = False
            if up_db == True:
                if re.search('tolerance', line):
                    if doubles['tolerance'] == None:
                        data = line.split('tolerance')
                        doubles['tolerance'] = data[1].strip() 
                        up_db = False
    
            # Information in the header
            for key in self.vals: 
                # check for things to set the flags
                if re.search('multilevel_quad_db', line):
                    ml_flag = True
                if re.search('upscatter_db', line):
                    up_db = True   
                # now look for the rest of the keys
                if re.search(key, line):
                    data = line.split(key)
                    self.vals[key] = data[1].strip()  
            
            # Time and CUDA information
            for key in time_vals:
                if re.search(key, line):
                    data = line.split(key)
                    num = data[1].strip().split()
                    self.time_vals[key] = num[0].strip()
            
            # Iteration information
            for key in self.iter_dict:
                if re.search(key, line):
                    data = line.split(key)
                    num = data[1].strip().split()
                    self.iter_dict[key].append(num[0].strip())
            # k; need separately since it's on the same line as iteration #
            for key in self.k:
                if re.search(key, line):
                    data = line.split(key)
                    num = data[1].strip().split()
                    self.k[key].append(num[0].strip())
                    
        # put k into the iter_dict to simplify printing later
        self.iter_dict['eigenvalue of'] = self.k['eigenvalue of']
        
        # calculate things we want in the spreadsheet
        self.vals['cells']   = int(self.vals['num cells I']) * int(self.vals['num cells J']) * int(self.vals['num cells K'])
        self.vals['blocks']  = int(self.vals['num_blocks_i']) * int(self.vals['num_blocks_j'])
        self.vals['domains'] = int(self.vals['blocks']) * int(self.vals['num_sets'])
        # set doubles
        self.vals['ml_Pn']  = ml_doubles['Pn_order']
        self.vals['ml_az']  = ml_doubles['azimuthals_octant']
        self.vals['ml_oct'] = ml_doubles['polars_octant']
        self.vals['up_tol'] = doubles['tolerance']

        
        
    def print_csv(self,name):
        """Print the read in data to name.csv"""

        # itereation keys we'd like to print since they have nicer names
        iter_keys = ['eigen_iters', 'rel_err_k', 'abs_err_k', 'fxd_iters']
        # How the keys map to the keys that we used on read-in from the file
        key_map = {'fxd_iters' : 'Iterations =', 'eigen_iters' : 'Eigenvalue Iteration   ',
                   'k' : 'eigenvalue of',  'rel_err_k' : 'relative error in k-eff of ', 
                   'abs_err_k' : 'absolute error in k-eff of '}

        with open(name+'.csv', 'wb') as csvfile:
            # write the header / problem specification information
            writer = csv.DictWriter(csvfile, fieldnames = self.keys, dialect='excel', 
                                    restval='',extrasaction='ignore')
            #if not csv.Sniffer().has_header(csvfile.read(1024)):
            writer.writeheader()
            writer.writerow(vals)
    
            # Now write timing information
            writer = csv.DictWriter(csvfile, fieldnames = self.time_keys, dialect='excel', 
                                    restval='',extrasaction='ignore')
            writer.writeheader()
            writer.writerow(time_vals)
    
            # And iteration information
            writer = csv.DictWriter(csvfile, fieldnames = iter_keys, dialect='excel', 
                                    restval='',extrasaction='ignore')
            writer.writeheader()
            length = len(self.iter_dict[key_map[iter_keys[0]]])
            for row_num in range(length):
                # get a new dictionary that will contain one row of data at each new row
                tmp_dict = {}
                for key in iter_keys:
                    tmp_dict[key] = self.iter_dict[key_map[key]][row_num]
                writer.writerow(tmp_dict)
                
# We want the k on the last iteration, the total number of fixed iters, last rel and abs err
# eigen solver, preconditioner data, wall time
    def highlights(self):
        """Print some highlights we care about most"""
        
        # Get total number of inner iterations
        self.iter_dict['Iterations ='] = map(int, self.iter_dict['Iterations ='])
        total_inner = np.sum(self.iter_dict['Iterations ='])
        
        # print
        print "After ",  self.time_vals['walltime'], "minutes,", \
        self.iter_dict['Eigenvalue Iteration   '][-1], "eigenvalue iterations, and", \
        total_inner, "total inner iterations,"
        print "k was", self.iter_dict['eigenvalue of'][-1], "with relative error",\
        self.iter_dict['relative error in k-eff of '][-1], "and absolute error",\
        self.iter_dict['absolute error in k-eff of '][-1]
        
        

### Process a bunch of files

In [58]:
# Name of the file containing the output we want to process 
file_location = '/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/'
file_name='bw1484_small.out' 
full_out_file = os.path.join(file_location, file_name)

# Get a DenovoProcessor, parse the file, and write to csv
processor = DenovoProcessor()
processor.parse_file(full_out_file)
processor.print_csv('bw1484_small')
processor.highlights()

After  7211 minutes, 59 eigenvalue iterations, and 2679 total inner iterations,
k was 1.0323147041e+00 with relative error 1.693561e-02 and absolute error 1.748288e-02
